## SET ENVIRONMENT

In [1]:
import os 
import pandas as pd 
import re
import itertools
import numpy as np

In [2]:
os.chdir(r'C:\Users\Adam\Python Scripts\feldolgozott_kepek')
mapping = pd.read_excel(r'C:\Users\Adam\Python Scripts\invoice_mapping\invoice_subitem_map.xlsx')

In [3]:
# SUBSET MAPPING 
unique_types = list(set(mapping['Type']))

itemized = pd.DataFrame()
itemized['Field_Name'] = []
itemized['File_Name'] = []
itemized['Value'] = []

app1 = pd.Series()
app2 = pd.Series()
app3 = pd.Series()

C:\Users\Adam\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
C:\Users\Adam\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Adam\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


## OCR 
https://nanonets.com/blog/ocr-with-tesseract/

In [4]:
from PIL import Image
from pytesseract import image_to_string
import os 
import pytesseract
import numpy as np
import re
import cv2

## Tesseract location 
# C:\Program Files\Tesseract-OCR
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [5]:
test = ['Type1']

In [7]:
os.chdir('C:\\Users\\Adam\\Python Scripts\\feldolgozott_kepek\\')
folders = os.listdir()

#LOOP THROUGH FOLDERS
#for i in range(len(folders)): 
for i in range(len(test)):
    print('OUTER LOOP:')
    print(i)
    
    os.chdir('C:\\Users\\Adam\\Python Scripts\\feldolgozott_kepek\\' + folders[i])
    print('#################################### WORKDIR:')
    print(os.getcwd())
    
    files = os.listdir() 
    files.remove('_szovegge_alakitott_kepek_')
    print('FILES IN DIR:')
    print(files)
    
    # CREATE FOLDER FOR TXT FILES
    directory = 'C:\\Users\\Adam\\Python Scripts\\feldolgozott_kepek\\' + folders[i] + '\\'+ '_teteles_szovegge_alakitott_kepek_'+'\\'
    if not os.path.exists(directory):
        os.makedirs(directory)
    else: 
        os.chdir(directory)
    print('################################### DIRECTORY TO BE CREATED:')
    print(directory)

    # LOOP THROUGH FILES IN FOLDER
    for f in range(len(files)): 
        file = files[f]
        im = Image.open(file)
        im = np.float32(im)
        #im = cv2.imread(file,0)

        print('################################### INNER LOOP:')
        print(f)
        
        # PREPROCESS IMAGE 

        scale_percent = 350 # percent of original size
        width = int(im.shape[1] * scale_percent / 100)
        height = int(im.shape[0] * scale_percent / 100)
        dim = (width, height)
        im = cv2.resize(im, dim )
        
        # get grayscale image
        #
        im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        
        # noise removal
        #im = cv2.medianBlur(im,3)
        
        # Tresholding
        #im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        #    cv2.THRESH_BINARY,11,2)

        
        kernel = np.ones((1, 1), np.uint8)
        im = cv2.dilate(im, kernel, iterations=5)
        #im = cv2.erode(im, kernel, iterations=1)
        

        # RUN TESSERACT ON FILE
        # text = pytesseract.image_to_string(im, lang = 'hun', config='-c preserve_interword_spaces=1' '--oem 3' )
        text = pytesseract.image_to_string(im, config='-l hun+eng -c preserve_interword_spaces=1'  '--oem 1' '--psm 6' )
 

        # WRITE OUTPUT TO NEW FILE 
        filename= directory+files[f][:-4] + 'txt'
        testfile = open(filename, "w")
        
        # convert multiple spaces into @
        #text = re.sub('\s\s+', ' @ ', text)
        text = re.sub('  +', ' @ ', text)


        testfile.write(text)
        testfile.close() 
        print('FILE CREATED:')
        print(filename)
        

OUTER LOOP:
0
#################################### WORKDIR:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type1
FILES IN DIR:
['t1_szamla_1.tiff', 't1_szamla_2.tiff', 't1_szamla_3.tiff', 't1_szamla_4.tiff', '_teteles_szovegge_alakitott_kepek_']
################################### DIRECTORY TO BE CREATED:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type1\_teteles_szovegge_alakitott_kepek_\


FileNotFoundError: [Errno 2] No such file or directory: 't1_szamla_1.tiff'

## CONVERTING OCR OUTPUT INTO PANDAS DATAFRAME

In [8]:
df_all = pd.DataFrame()

## LOOP I  - go through each Invoice Type ######################################
for m in range(len(unique_types)):
    
    # SUBSET MAPPING PER TYPE 
    
    # printings 
    print('########################   1 - TYPE RUNNING:   ################################')
    
    applicable_mapping = mapping.loc[mapping['Type'] == unique_types[m]]
    # setting wd for each mapping
    dry = r'C:\Users\Adam\Python Scripts\feldolgozott_kepek' + '\\' + unique_types[m] +'\\' + '_teteles_szovegge_alakitott_kepek_'
    os.chdir(dry)

    text_files = os.listdir()
    
    ## LOOP II  - go thorugh each range ######################################
    #for f in range(len(set(mapping['Starting_range']))):
    for f in range(len(text_files)):
        
        # printings 
        print(' ############################### 2 - RANGE RUNNING ###############################:')  

        # READ IN FILE 
        
        text = open(text_files[f]).read()

        # STARTING RANGE
        lookup = applicable_mapping['Starting_range'][f]
        with open(text_files[f]) as myFile:
            for num, line in enumerate(myFile, 1):
                if lookup in line:
                    print('found at line:'), print(num)
                    start_range = num -1
        
        # ENDING RANGE
        lookup = applicable_mapping['Ending_range'][f]
        with open(text_files[f]) as myFile:
            for num, line in enumerate(myFile, 1):
                if lookup in line:
                    print('found at line:'), print(num)
                    end_range = num -1
                    
        
        ## READ RANGE ONLY 
        text_file = open(text_files[f], "r")
        lines = text_file.readlines()
        lines = lines[start_range:end_range]
        

        # remoe lines starting with char 
        char = mapping['Remove_lines_containing'][0]

        s = list()

        for line in lines :
             if char not in line and char not in line:
                    #print(line)
                    s.append(line)


        # convert into list 
        s = list(map(str.strip, s))

        # remove empty lines 
        s = [x for x in s if x]
        
        print('###################################### 3 - S Created  ###############################: '), print(s)

        
        for i in range(len(s)):
            print('')
            # PRINT LOOP LENGTH AND STATUS 
            print('LOOP LENGTH:'), print(len(s))
            print('LOOP AT:'), print(i)

            # FORMAT DATA 
            t= pd.Series(s[i]).str.split(pat = '@')
            t = t.astype(str).values.tolist()
            t = [x for y in t for x in y.split("', '")]
    
            print('###################################### 4 - T Created  ###############################: '), print(t)

            lst = list()
            lst.append(text_files[f])
            df = pd.DataFrame()


            for r in range(len(mapping['Starting_range'])):
                ''''''
                # ITEM NAVIGATOR 
                nav_start = mapping['Start'].values[r]
                nav_stop = mapping['Stop'].values[r]
                nav_step = mapping['Step'].values[r]
                nav = slice(int(nav_start), int(nav_stop), int(nav_step))

                # items
                t2= t[nav] #.copy()

                print('###################################### 4.Append - T2 append Created  ###############################: '), print(t)
                
                lst.append(t2)

            # CREATING DATAFRAME 

            # remove empty lines from list 
            lst = [x for x in lst if x]

            print('PRINTING LST HERE: '), print(lst)

            # convert list to dataframe 
            df = pd.DataFrame(lst)

            # define the number of columns needed in DF 
            cols = int(len(mapping['Start']))

            # calculate the length of desired DF 
            df_len = int(len(df)/cols) 

            # create DF 
            df = pd.DataFrame(np.reshape(df.values,(df_len,cols+1))) # adding +1 as we need to add 1 column for file name 
            print('############################## 6. - Creating DF #############################'), print(df)
            
            df_all = df_all.append(df)


########################   1 - TYPE RUNNING:   ################################
 ############################### 2 - RANGE RUNNING ###############################:
found at line:
24
found at line:
31
###################################### 3 - S Created  ###############################: 
['Megnevezés @ Vtsz/Teszor . Egységár — Mennyiség Afa(%) @ Nettó @ Áfa @ Bruttó', 'Kiszállási díj @ 0 @ 8100,00Ft 1,00alkalom 27% @ 8 100,00 Ft @ 2 187,00 Ft @ 10 287,00 Ft', 'Tanácsadás @ 2525 @ 13 500,00 Ft @ 4,00 óra @ 27% @ 54 000,00 Ft @ 14 580,00 Ft @ 68 580,00 Ft']

LOOP LENGTH:
3
LOOP AT:
0
###################################### 4 - T Created  ###############################: 
["['Megnevezés ", ' Vtsz/Teszor . Egységár — Mennyiség Afa(%) ', ' Nettó ', ' Áfa ', " Bruttó']"]
###################################### 4.Append - T2 append Created  ###############################: 
["['Megnevezés ", ' Vtsz/Teszor . Egységár — Mennyiség Afa(%) ', ' Nettó ', ' Áfa ', " Bruttó']"]
#########################

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 68: character maps to <undefined>

In [9]:
df_all = df_all.reset_index(drop=True)

In [10]:
# https://stackoverflow.com/questions/49997911/relabelling-column-from-rangeindex-in-pandas-dataframe?noredirect=1&lq=1

In [11]:
cdf = pd.DataFrame()

for i in range(len(df_all.columns)): 
    df = df_all[i]
    
    for e in range(len(df)):
        df[e] = re.sub(r'[^\w]', ' ', str(df[e]))
        df[e] = df[e].lstrip()
        df[e] = df[e].rstrip()

        df_all[i][e] = df[e]

In [12]:
df_all.columns = df_all.iloc[0]

In [14]:
df_all = df_all[df_all['Megnevezés'] != 'Megnevezés']

In [21]:
os.chdir('C:\\Users\\Adam\\Python Scripts\\Output')

In [22]:
df_all.to_excel('sub_items.xlsx')